In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

### 0 - Read and process collected driver data
- Just including all drivers output from Linx here

In [ ]:
som_exome <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/somatic_exome.csv")

In [ ]:
#dnds <- readRDS("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/dnds/dnds.Rds")
sel_cvs <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/dnds/sel_cvs.csv")

In [ ]:
purities <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/purities.csv")

In [ ]:
cn_gene <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/cn_gene.csv")

In [ ]:
breakends <- fread( paste0(TMP_DIR, "structural_variants/breakend.csv")) 

In [ ]:
drivers <- fread( paste0(TMP_DIR, "drivers.csv"))

In [ ]:
#head(som_exome)

#### Check DNDS

In [ ]:
tmp <- 
sel_cvs %>% 
 mu(ct = n_syn + n_mis + n_non + n_spl + n_ind, 
    ratio = wnon_cv/wmis_cv, 
    type = ifelse(ratio > 1, "TSG", "ONC")) %>% 
 fi(cohort == "Pan-Cancer") %>% 
 se(cohort, gene_name, ct, type, ratio, contains("n_"), contains("w"), pglobal_cv, qglobal_cv) %>% 
 ar(desc(ct))

In [ ]:
tmp %>% ar(qglobal_cv) %>% fi(type == "ONC", grepl("FGF", gene_name)) %>% ar(ratio)

In [ ]:
pan <- tmp %>% fi(cohort == "Pan-Cancer") %>% pu(gene_name) %>% unique()

In [ ]:
sel_cvs %>% 
 fi(gene_name == "BMP5") %>% ar(pglobal_cv) %>% se(cohort, gene_name, contains("n_"), contains("w"), pglobal_cv) 

In [ ]:
tmp %>% fi(!gene_name %in% pan, ct > 10) %>% ar(pglobal_cv)

In [ ]:
dnds$globaldnds

In [ ]:
see <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/drivers.csv")

In [ ]:
drivers <- 
fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/drivers.csv") %>% 
 fi(likelihoodMethod == "DNDS") %>% 
 gb(gene) %>% 
 su(like = mean(driverLikelihood > .8), ct = n()) %>% mu(gene_name = gene) %>% 
 ug()

In [ ]:
see %>% 
 gb(gene, likelihoodMethod) %>% 
 su(ct = n()) %>% 
 ar(desc(ct)) %>% 
 fi(gene == "PIK3CA")

In [ ]:
dnds$sel_cv %>% ij(drivers, by = "gene_name") %>% se(gene_name, contains("n_"), contains("w"), like, ct, pglobal_cv) %>% gb(pglobal_cv < .01) %>% su(ct = n())

In [ ]:
dnds$sel_cv %>% fi(qglobal_cv < .25) %>% fi(n_syn != 0)

### Somatic Drivers 

- Read in prepped exome file

In [ ]:
somatic_exome <- 
som_exome %>% 
 gb(chromosome, position, gene, tier) %>% 
 mu(ct = n(), hotspot = (tier == "HOTSPOT" | ct > 30))

- Hotspots

In [ ]:
hotspots <- 
somatic_exome %>% 
 fi(hotspot) %>% 
 tm(sampleId, chromosome, gene, transcript, annotation, biallelic, method = "HOTSPOT")

- Biallelic

In [ ]:
biallelic <- 
somatic_exome %>% 
 fi(!hotspot, biallelic) %>% 
 tm(sampleId, chromosome, gene, transcript, annotation, biallelic, method = "BIALLELIC")

- Inframe

In [ ]:
inframe <- 
somatic_exome %>% 
 fi(!hotspot, !biallelic, annotation %in% c("inframe_deletion", "inframe_insertion", "phased_inframe_insertion", "phased_inframe_deletion")) %>% 
 tm(sampleId, chromosome, gene, transcript, annotation, biallelic, method = "INFRAME")

- Somatic drivers 

In [ ]:
som_drivers <- rbind(hotspots, biallelic, inframe) %>% lj(cn_gene %>% se(gene, chromosomeBand), by = "gene")

In [ ]:
#head(som_drivers)

- Now DNDS! 

In [ ]:
#dnds 

### Copy number drivers 

In [ ]:
amps <- 
cn_gene %>% 
 ij(purities %>% se(sampleId, ploidy), by = "sampleId") %>%
 fi(maxCopyNumber > (3*ploidy)) %>% 
 mu( method = ifelse(minCopyNumber > 3*ploidy, "AMP", "PARTIAL_AMP"))

In [ ]:
dels <- 
cn_gene %>% 
 ij( purities %>% se(sampleId, ploidy), by = "sampleId" ) %>%
 fi( minCopyNumber < .5 ) %>% 
 mu( method = "DEL" )  

In [ ]:
length(unique(cn_gene$sampleId))

In [ ]:
head(dels)

In [ ]:
drivers %>% fi(grepl("TRAF3", gene))

In [ ]:
dels %>% gb(chromosome, chromosomeBand, gene) %>% su(ct = n()) %>% ar(desc(ct)) %>% fi(chromosome == 23)

### Structual Variant Drivers

- Disruptions

In [ ]:
disruptions <- 
breakends %>%
 fi(disruptive) %>% 
 tm(sampleId, chromosome, chromosomeBand = chrBand, gene, transcriptId, canonical, disruptive, method = "DISRUPTION") %>% 
 unique()

In [ ]:
head(disruptions)

In [ ]:
disruptions %>% 
 gb(chromosome, chromosomeBand, gene) %>% 
 su(ct = n()) %>% 
 ar(desc(ct))